In [13]:
pip install pyowm --quiet


Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install meteostat --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install openmeteo --quiet


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install requests-cache retry-requests numpy pandas --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyowm
from pyowm import OWM

In [16]:
import meteostat

import requests_cache
from retry_requests import retry

ModuleNotFoundError: No module named 'requests_cache'

In [ ]:


# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2025-04-28",
	"end_date": "2025-05-12",
	"daily": ["temperature_2m_mean", "temperature_2m_max", "temperature_2m_min", "daylight_duration", "precipitation_sum", "rain_sum", "wind_speed_10m_max"],
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "surface_pressure"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["surface_pressure"] = hourly_surface_pressure

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(3).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(4).ValuesAsNumpy()
daily_rain_sum = daily.Variables(5).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(6).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

In [ ]:
owm = OWM('1c9881707b4f4d82c9bdbb1e2a5296f4')
mgr = owm.weather_manager()

observation = mgr.weather_at_place(input("Enter the city and country short form: "))
w = observation.weather

print(w.detailed_status)         
print(w.wind())                  
print(w.humidity)               
print(w.temperature('celsius'))  
print(w.rain)                    
print(w.heat_index)              
print(w.clouds)

clear sky
{'speed': 0.45, 'deg': 153, 'gust': 2.24}
66
{'temp': 16.69, 'temp_max': 17.31, 'temp_min': 14.99, 'feels_like': 16.14, 'temp_kf': None}
{}
None
0
